In [1]:
import pandas as pd
import requests
import json

operations_ids = []


prev_titles = []
news_dataset = pd.read_csv('Lenta_dataset.csv')
news_dataset.head()

,Unnamed: 0,url,title,text,topic,tags,date
0,0,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,NaN
1,1,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,NaN
2,2,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,NaN
3,3,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина,NaN
4,4,https://lenta.ru/news/2018/12/15/skrepy/,Российскую молодежь предложили обучать духовны...,Российская молодежь лучше усвоит духовные ценн...,Интернет и СМИ,Coцсети,NaN


In [4]:
news_dataset.tags.unique()

array(['Мир', 'Политика', 'Общество', 'Украина', 'Coцсети',
       'Следствие и суд', 'Хоккей', 'Деньги', 'Госэкономика',
       'Происшествия', 'Футбол', 'Зимние виды', 'Бизнес', 'Музыка',
       'Квартира', 'Бокс и ММА', 'Театр', 'Оружие', 'Дача', 'Прибалтика',
       'Рынки', 'Звери', 'Техника', 'Интернет', 'Люди', 'Наука',
       'Внешний вид', 'Кино', 'ТВ и радио', 'Регионы', 'Космос',
       'Явления', 'Стиль', 'Белоруссия', 'События', 'Деловой климат',
       'Все', 'Искусство', 'Движение', 'Преступность', 'Криминал',
       'Жизнь', 'Мнения', 'Полиция и спецслужбы', 'Конфликты', 'Авто',
       'Книги', 'Вкусы', 'Фотография', 'Реклама', 'Закавказье', nan,
       'Часы', 'Летние виды', 'Россия', 'Пресса', 'Вещи', 'История',
       'Игры', 'Достижения', 'Офис', 'Средняя Азия', 'Еда', 'Москва',
       'Социальная сфера', 'Молдавия', 'Преступная Россия', 'Гаджеты',
       'Город', 'Киберпреступность', 'Страноведение', 'Софт',
       'Инструменты', 'Вирусные ролики', 'Экология', 'Выб

In [5]:
already_have = pd.read_csv('dataset_generated_to_not.csv')
already_have_titles = already_have.title

In [6]:
safe_df = news_dataset[(news_dataset['tags'].isin(['Пресса', 'Вещи', 'История',
       'Игры', 'Достижения', 'Офис', 'Мемы',
       'Наследие', 'Туризм', 'Страноведение', 'Софт',
       'Инструменты', 'Вирусные ролики', 'Экология', 'Часы', 'Звери', 'Техника', 'Интернет', 'Люди', 'Наука',
       'Внешний вид', 'Кино', 'ТВ и радио', 'Летние виды', 'Средняя Азия', 'Еда', 'Авто',
       'Книги', 'Вкусы', 'Фотография', 'Реклама'])) & (~news_dataset.title.isin(already_have_titles))]
safe_df

,Unnamed: 0,url,title,text,topic,tags,date
66,66,https://lenta.ru/news/2018/12/14/kot_putina/,Кот Путина освоил японский язык,"Сибирский кот Мир, подаренный российским лидер...",Из жизни,Звери,NaN
68,68,https://lenta.ru/news/2018/12/14/perfect_hell/,В Windows 10 нашли ад перфекциониста,После очередного обновления операционной систе...,Наука и техника,Техника,NaN
69,69,https://lenta.ru/news/2018/12/14/microsoft/,Обнаружен способ взломать любую учетную запись...,Исследователь в области кибербезопасности из И...,Интернет и СМИ,Интернет,NaN
72,72,https://lenta.ru/news/2018/12/14/save/,Полицейский поймал выпрыгнувшего из горящего д...,"В США полицейский сумел поймать мальчика, выпр...",Из жизни,Люди,NaN
75,75,https://lenta.ru/news/2018/12/14/dry/,Предсказана глобальная нехватка питьевой воды,Ученые Университета Нового Южного Уэльса (Авст...,Наука и техника,Наука,NaN
...,...,...,...,...,...,...,...
161258,161258,https://lenta.ru/news/2016/01/25/orgy/,Семейные оргии с проститутками довели еврейску...,Супружеская пара из Израиля обратилась в равви...,Из жизни,Люди,NaN
161260,161260,https://lenta.ru/news/2016/01/25/reuters/,Опубликован список самых влиятельных в мире уч...,Компания Thomson Reuters представила отчет «Са...,Наука и техника,Наука,NaN
161261,161261,https://lenta.ru/news/2016/01/25/breguet/,Мануфактура Breguet представила новые женские ...,Швейцарская часовая мануфактура Breguet предст...,Ценности,Часы,NaN
161263,161263,https://lenta.ru/news/2016/01/25/dino/,Житель США на спор вышел разгребать снег в кос...,Житель американского штата Кентукки Клифф Уайт...,Из жизни,Люди,NaN


In [7]:
generated_result = pd.DataFrame(columns=['title', 'generated_text'])
generated_result.head()

,title,generated_text


In [58]:
now_titles = safe_df.title.sample(500)
titles =  [x for x in now_titles if x not in prev_titles]
prev_titles = prev_titles + titles 
titles

['Американка удалила восемь килограммов кожи ради идеальной свадьбы',
 'Звезда «Грязных танцев» отказалась сняться в\xa0ремейке',
 'Названа дата выхода новой Windows',
 'The Sun рассказала о\xa0подготовке Тома Круза к\xa0последнему полету шаттла «Колумбия»',
 'Водителя Панина наказали за\xa0езду по\xa0пешеходной улице Саратова',
 'Пилот Ferrari Феттель выиграл квалификацию Гран-при России',
 'Любительница селфи уничтожила работы Саймона Бирча на\xa0200\xa0тысяч долларов',
 'Раскрыта тайна «отворившегося неба» над Астраханью',
 '«Смертельное оружие» и «Изгоняющего дьявола» превратят в\xa0сериалы',
 'Австралийка забеременеет от\xa0мертвого партнера',
 'Любимый модельер Мишель Обамы переключился на\xa0косметику',
 'Подростки голыми руками вырвали пса из\xa0смертельных объятий змеи',
 'Рэпер Face побывал в\xa0гостях на\xa0Comedy Club',
 'Школьник «романтично» пошутил про чернокожих и\xa0разгневал пользователей сети',
 'Актер Сергей Гармаш попал в\xa0аварию',
 'Определен произведенный на\xa

In [7]:
len(titles)

400

In [ ]:
token = "IAM_token"

In [59]:
def generate_text(title):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync"
    headers = {
           f"Authorization": "Bearer {token}",
           "Content-Type": "application/json"
       }
    data = {
  "modelUri": "gpt://b1gl2mq1047r4bitl8nj/yandexgpt/latest",
  "completionOptions": {
    "stream": True,
    "temperature": 0.7,
    "maxTokens": 1000
  },
  "messages": [
    {
      "role": "system",
      "text": "Ты журналист новостного портала. Тебе необходимо написать статью на заданную тему."
    },
    {
      "role": "user",
      "text": f"Напиши новостную статью с заголовком: {title}"
    }
  ]
}
    response = requests.post(url, headers=headers, data=json.dumps(data))

    return response.json()


for i in titles:
    operation = generate_text(i)
    operations_ids.append((i, operation['id']))   


In [89]:
len(operations_ids)

58

In [111]:
generated = []
for operation in operations_ids[::]:
    response = requests.get(f"https://llm.api.cloud.yandex.net/operations/{operation[1]}", headers={"Authorization": f"Bearer {token}"})
    answ = response.json()
    try:
        generated.append((operation[0], answ['response']['alternatives'][0]['message']['text']))
        operations_ids.remove(operation)
    except:
        print('.')
        pass

.


In [112]:
len(operations_ids)

1

In [113]:
tmp = pd.DataFrame(generated, columns=['title','generated_text'])
generated_result = pd.concat([generated_result, tmp])
generated_result=generated_result.drop_duplicates().reset_index(drop=True)

In [114]:
generated_result

,title,generated_text
0,В Cоюзе журналистов осудили ведущего НТВ Норки...,В Союзе журналистов России осудили ведущего НТ...
1,Норильский инженер напечатал дешевый бионическ...,**Норильский инженер напечатал дешёвый биониче...
2,Возобновлена веб-трансляция из вольеров Тимура...,**Возобновлена веб-трансляция из вольер Тимура...
3,Новосибирец за год прослушал треки Radiohead 2...,### Новосибирец за год прослушал треки Radiohe...
4,В сети высмеяли клон пакета из IKEA за две тыс...,**В сети высмеяли клон пакета из IKEA за две т...
...,...,...
2528,Взломавший городские сирены хакер перебудил ты...,**Взломавший городские сирены хакер перебудил ...
2529,Шведы нашли способ удалиться из интернета,**Шведы нашли способ удалиться из интернета**\...
2530,В Казани стартовала конференция «Наука будущего»,**В Казани стартовала конференция «Наука будущ...
2531,В ЮАР сняли на видео испуганную угрюмую лягушку,"В ЮАР сняли на видео лягушку, которая выглядел..."


In [115]:
generated_result.to_csv('generated_YaGPT.csv')